In [1]:
import numpy as np
import cv2 as cv2
import os
from tqdm import tqdm

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation, Dropout
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import itertools

import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\mahsa\AppData\Local\conda\conda\envs\python3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_path = '/Users/mahsa/OneDrive/Desktop/ML-projects/dogs-cats/train/'
test_path =  '/Users/mahsa/OneDrive/Desktop/ML-projects/dogs-cats/test/'

In [3]:
img_width = 200
img_height = 200

def prepare_data(list_of_images):
   
    image_data = [] # images 
    labels = [] # labels

    for image in tqdm(list_of_images):
        img = cv2.resize(cv2.imread(image), (img_width,img_height), interpolation=cv2.INTER_CUBIC)
        image_data.append(img)
        
    for i in list_of_images:
        if 'dog' in i:
            labels.append([1,0])
        elif 'cat' in i:
            labels.append([0,1])
                
    return image_data, labels


In [4]:
train_images= [train_path + i for i in os.listdir(train_path)] 
test_images = [test_path + i for i in os.listdir(test_path)]

X, Y = prepare_data(train_images[:100])
#test_data = prepare_data(test_images)

X_train, X_val, Y_train, Y_val = train_test_split(np.array(X), np.array(Y), test_size=0.2, random_state=121)

100%|██████████| 100/100 [00:00<00:00, 378.15it/s]


In [5]:
len(X_train)

80

In [6]:
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
batch_size = 5

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
   )

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
   )

train_generator = train_datagen.flow(X_train, Y_train, batch_size=batch_size)
validation_generator = val_datagen.flow(X_val, Y_val, batch_size=batch_size)

In [7]:
n_classes =2

model = Sequential()

model.add(Conv2D(8, kernel_size=(3, 3), strides=(1, 1),
                 activation='relu',
                 input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 8)       224       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 99, 99, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 97, 97, 16)        1168      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               7373000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
__________

In [8]:
model.fit_generator(train_generator, steps_per_epoch= batch_size * len(X_train), 
                    validation_data=validation_generator, 
                    validation_steps=batch_size * len(X_val), epochs=5, verbose=2)

Epoch 1/5
 - 118s - loss: 0.0024 - acc: 0.9995 - val_loss: 9.1196e-07 - val_acc: 1.0000
Epoch 2/5
 - 113s - loss: 9.3170e-07 - acc: 1.0000 - val_loss: 2.0683e-07 - val_acc: 1.0000
Epoch 3/5
 - 118s - loss: 7.0176e-07 - acc: 1.0000 - val_loss: 1.2803e-07 - val_acc: 1.0000
Epoch 4/5
 - 115s - loss: 4.8051e-07 - acc: 1.0000 - val_loss: 1.2493e-07 - val_acc: 1.0000
Epoch 5/5
 - 124s - loss: 2.0939e-07 - acc: 1.0000 - val_loss: 1.2302e-07 - val_acc: 1.0000


In [9]:

# batch_size = 32 # in each iteration, we consider 32 training examples at once
# num_epochs = 200 # we iterate 200 times over the entire training set
# kernel_size = 3 # we will use 3x3 kernels throughout
# pool_size = 2 # we will use 2x2 pooling throughout
# conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
# conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
# drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
# drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
# hidden_size = 512 # the FC layer will have 512 neurons   validation_steps=batch_size * len(X_val), epochs=5, verbose=2)



# inp = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
# # Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
# conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
# conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
# pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
# drop_1 = Dropout(drop_prob_1)(pool_1)
# # Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
# conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
# conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
# pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
# drop_2 = Dropout(drop_prob_1)(pool_2)
# # Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
# flat = Flatten()(drop_2)
# hidden = Dense(hidden_size, activation='relu')(flat)
# drop_3 = Dropout(drop_prob_2)(hidden)
# out = Dense(num_classes, activation='softmax')(drop_3)

# model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers